## Requirements

In [1]:
# %pip install firebase_admin geopy

## Imports

In [2]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
from google.cloud.firestore import GeoPoint
from google.cloud.firestore_v1.base_query import FieldFilter

import pytz
import numpy as np
from pprint import pprint 
from itertools import combinations
from geopy.distance import geodesic
from scipy.spatial import ConvexHull
from datetime import datetime, timedelta


## Params

In [3]:
thres_distance = 1000
creds_path = "./keys/serviceAccountKey.json"
indian_timezone = pytz.timezone('Asia/Kolkata')
DEBUG = True

## utils

In [4]:
def calculate_convex_hull_area(coords):
    hull = ConvexHull(coords)
    return hull.volume  # For 2D convex hull, use hull.area

In [5]:
def select_bounding_coords(coordinates:list)->list:
    max_area = 0
    max_area_coords = None

    # Iterate through all combinations of coordinates
    for i in range(len(coordinates), -1,-1):
        for comb in combinations(coordinates, i):        # Assuming you want to form a triangle (change to 4 for quadrilateral, etc.)
            if not all(c in comb for c in coordinates):  # Skip combinations where not all coordinates are included
                continue
            area = calculate_convex_hull_area(np.array(comb))
            if area > max_area:
                max_area = area
                max_area_coords = comb

    coordList = [GeoPoint(x[0], x[1]) for x in max_area_coords]
    
    return coordList

In [6]:
def get_dict_by_regionMapId(regionMap_id):
    for region_map in regionMaps:
        if region_map['regionMap_id'] == regionMap_id:
            return region_map
    return None

In [7]:
def get_marker_cord_by_id(marker_id):
    for marker in markers_:
        if marker['marker-id'] == marker_id:
            return GeoPoint(marker['marker_cord'][0],marker['marker_cord'][1])
    return None

In [8]:
def get_marker_cord_by_id_tuple(marker_id):
    for marker in markers_:
        if marker['marker-id'] == marker_id:
            return marker['marker_cord']
    return None

In [9]:
# if DEBUG:
#     import matplotlib.pyplot as plt
#     x, y = zip(*coordinates)
#     plt.scatter(x, y, color='blue', label='Original Coordinates')
    
#     # Plot the convex hull with maximum area
#     hull = ConvexHull(np.array(max_area_coords))
#     for simplex in hull.simplices:
#         plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    
#     plt.scatter(hull.points[hull.vertices, 0], hull.points[hull.vertices, 1], color='red', label='Convex Hull Vertices')
    
#     plt.title('Convex Hull with Maximum Area')
#     plt.xlabel('Latitude')
#     plt.ylabel('Longitude')
#     plt.legend()
#     plt.show()

## Connect

In [10]:
cred = credentials.Certificate(creds_path)
firebase_admin.initialize_app(cred)
db = firestore.client()

In [11]:
db

## Markers

In [12]:
current_datetime = datetime.now()
midnight_datetime = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

one_day = timedelta(days=1)
yesterday_midnight_datetime = current_datetime - one_day
yesterday_midnight_datetime = yesterday_midnight_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

start_date = indian_timezone.localize(yesterday_midnight_datetime)
end_date = indian_timezone.localize(midnight_datetime)


In [13]:
# # Set the start date to midnight of January 1, 2023
# start_date = datetime(2023, 1, 1, tzinfo=indian_timezone)
# start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

# # Set the end date to the current midnight
# midnight_datetime = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
# end_date = indian_timezone.localize(midnight_datetime)

In [14]:
markers_ = []
markers_get = db.collection('Markers').where(filter=FieldFilter('time', ">=", start_date)).where(filter=FieldFilter('time', "<", end_date)).get()
# markers_get = db.collection('Markers').get()
for m in markers_get:
    marker = {
        "marker-id":m.to_dict()["id"],
        "marker_cord":(m.to_dict()["lat"],m.to_dict()["long"]),
        }
    markers_.append(marker)

In [15]:
if DEBUG:
    pprint(markers_)

[{'marker-id': 1702831179224, 'marker_cord': (26.5276161, 93.885418)},
 {'marker-id': 1702831353978,
  'marker_cord': (26.523327432461524, 93.88401668518782)},
 {'marker-id': 1702831370550,
  'marker_cord': (26.525289944991446, 93.88164695352316)},
 {'marker-id': 1702837908940,
  'marker_cord': (26.528784706149324, 93.8829005509615)},
 {'marker-id': 1702840841383,
  'marker_cord': (26.300955599413115, 93.96943598985672)},
 {'marker-id': 1702856129469,
  'marker_cord': (24.83848107736667, 92.83123590052128)},
 {'marker-id': 1702966788552,
  'marker_cord': (26.1060139519425, 91.59353524446487)},
 {'marker-id': 1703030904099,
  'marker_cord': (24.81946022402917, 92.81988814473152)},
 {'marker-id': 1703059580160,
  'marker_cord': (24.830493344236597, 92.83261924982071)},
 {'marker-id': 1703486380661,
  'marker_cord': (26.140288349728852, 91.76433496177197)},
 {'marker-id': 1703667027697, 'marker_cord': (26.1988244, 91.6964265)},
 {'marker-id': 1703693608553, 'marker_cord': (19.0920725, 72.

## Region Maps

In [16]:
regionMaps = []

In [17]:
regionMaps_get = db.collection('RegionMap').get()
for m in regionMaps_get:
    data = m.to_dict()
    data["regionMap_id"]= m.id
    regionMaps.append(data)

In [18]:
if DEBUG:
    pprint(regionMaps)

[]


In [19]:
for r in regionMaps:
    r['central_coord'] = (r['central_coord'].to_protobuf().latitude, r['central_coord'].to_protobuf().longitude)
    r['coords'] = [(x.to_protobuf().latitude, x.to_protobuf().longitude)for x in r['coords']]

In [20]:
if DEBUG:
    pprint(regionMaps)

[]


## check for which region map is the marker

In [21]:
def is_within_distance(coord1, coord2, max_distance=thres_distance):
    # Calculate distance in meters using geopy
    distance = geodesic(coord1, coord2).meters
    return distance <= max_distance

In [22]:
parings = {}

In [23]:
for marker in markers_:
    marker_coord = marker['marker_cord']
    for region_map in regionMaps:
        for coord in region_map['coords']:
            if is_within_distance(marker_coord, coord):
                parings[marker['marker-id']]=region_map['regionMap_id']
                

In [24]:
if DEBUG:
    pprint(parings)

{}


In [25]:
for key in parings.keys():
    result_dict = get_dict_by_regionMapId(parings[key])
    ref = db.collection("RegionMap").document(parings[key])
    if DEBUG:
        print(ref.get().to_dict())
    result_dict["coords"].append(get_marker_cord_by_id_tuple(key))
    
    try:
        list_bbox = select_bounding_coords(result_dict["coords"])

        not_list_box = [GeoPoint(x[0],x[1])  for x in result_dict["coords"] if GeoPoint(x[0],x[1]) not in list_bbox]
    except Exception as e:
        if DEBUG:
            print(e)
        not_list_box= []
        
    if DEBUG:
        pprint(not_list_box)
    
    average_latitude = sum(lat for lat, lon in result_dict["coords"]) / len(result_dict["coords"])
    average_longitude = sum(lon for lat, lon in result_dict["coords"]) / len(result_dict["coords"])

    central_coord = GeoPoint(average_latitude, average_longitude)
    ref.update({
        "central_coord": central_coord,
        "coords": firestore.ArrayUnion([get_marker_cord_by_id(key)]),
        "markers": firestore.ArrayUnion([key])
        })
    if len(not_list_box)>0:
        ref.update({
            "coords": firestore.ArrayRemove(not_list_box),
            })

## remove updated markers

In [26]:
filtered_markers = [marker for marker in markers_ if marker['marker-id'] not in parings.keys()]

In [27]:
if DEBUG:
    pprint(filtered_markers)

[{'marker-id': 1702831179224, 'marker_cord': (26.5276161, 93.885418)},
 {'marker-id': 1702831353978,
  'marker_cord': (26.523327432461524, 93.88401668518782)},
 {'marker-id': 1702831370550,
  'marker_cord': (26.525289944991446, 93.88164695352316)},
 {'marker-id': 1702837908940,
  'marker_cord': (26.528784706149324, 93.8829005509615)},
 {'marker-id': 1702840841383,
  'marker_cord': (26.300955599413115, 93.96943598985672)},
 {'marker-id': 1702856129469,
  'marker_cord': (24.83848107736667, 92.83123590052128)},
 {'marker-id': 1702966788552,
  'marker_cord': (26.1060139519425, 91.59353524446487)},
 {'marker-id': 1703030904099,
  'marker_cord': (24.81946022402917, 92.81988814473152)},
 {'marker-id': 1703059580160,
  'marker_cord': (24.830493344236597, 92.83261924982071)},
 {'marker-id': 1703486380661,
  'marker_cord': (26.140288349728852, 91.76433496177197)},
 {'marker-id': 1703667027697, 'marker_cord': (26.1988244, 91.6964265)},
 {'marker-id': 1703693608553, 'marker_cord': (19.0920725, 72.

## create region map

In [28]:
coordinates = [entry['marker_cord'] for entry in filtered_markers]

# Create a list to store the clusters of coordinates
coordinate_clusters = []

# Iterate through each coordinate
for coord in coordinates:
    # Check if the coordinate is within 500 meters of any existing cluster
    found_cluster = False
    for cluster in coordinate_clusters:
        if any(is_within_distance(coord, c) for c in cluster):
            cluster.append(coord)
            found_cluster = True
            break

    # If not within 500 meters of any existing cluster, create a new cluster
    if not found_cluster:
        coordinate_clusters.append([coord])

## Add new region maps

In [29]:
region_data = []
for coords_lst in coordinate_clusters:
    marker_ids = [entry['marker-id'] for entry in markers_ if entry['marker_cord'] in coords_lst]
    
    average_latitude = sum(lat for lat, lon in coords_lst) / len(coords_lst)
    average_longitude = sum(lon for lat, lon in coords_lst) / len(coords_lst)

    central_coord = GeoPoint(average_latitude, average_longitude)

    data = {
        "central_coord": central_coord,
        "coords": [GeoPoint(x[0],x[1]) for x in coords_lst],
        "markers":marker_ids
    }
    region_data.append(data)

In [30]:
# create new region_maps
for data in  region_data:
    update_time, rm_ref = db.collection('RegionMap').add(data)
    if DEBUG:
        print(f"Added document with id {rm_ref.id}")

Added document with id J4DNGAlfSqnH7qhuR7Fh


Added document with id kIhQZt21ae9jm1480LsP
Added document with id pjhOHa22Cz9Sd5Wy71Kg
Added document with id AqCpIhirNtY4kdRqbR6X
Added document with id PXPTzKrff1444KtM38Hf
Added document with id vCJKnTmaEim118pSF49m
Added document with id Wld7RXbdOSl4rxKlDQIO
Added document with id TebtxX4DADvliq1CSuv2
Added document with id ovsxvVbGQBW5s0fkAc7N
Added document with id GtyXOxwKkxwDZGdDw5ZN
Added document with id r3BBlp5xYesx0AaSPufn
Added document with id OR12SogUL1T0Tga5yBga
Added document with id CYcbajvxUuW2zw3nlA7t
Added document with id DWt6p1oP9tZm8o6jJZL4
